In [1]:
import pandas as pd
from glob import glob
import numpy as np

In [2]:
sumrule_db = '../../Datasets/SumRuleCyl-allInOne/*equilProps-whole.csv'
sumrule_db = glob(sumrule_db)
print(sumrule_db)
equil = pd.read_csv(sumrule_db[0])
print(equil.columns)

['../../Datasets/SumRuleCyl-allInOne/allInOne-SumRuleCyl-bug-equilProps-whole.csv']
Index(['lineage_name', 'whole', 'ensemble_long', 'ensemble', 'space', 'nmon',
       'epsilon', 'dcyl', 'lcyl', 'dcrowd', 'ncrowd', 'dt', 'bdump', 'adump',
       'ensemble_id', 'dmon', 'mmon', 'eps_others', 'mcrowd', 'dwall',
       'phi_m_bulk', 'rho_m_bulk', 'phi_c_bulk', 'rho_c_bulk', 'n_frames',
       'asphericityMon-mean', 'asphericityMon-var', 'asphericityMon-sem',
       'fsdMon-mean', 'fsdMon-var', 'fsdMon-sem', 'gyrMon-mean', 'gyrMon-var',
       'gyrMon-sem', 'rfloryMon-mean', 'rfloryMon-var', 'rfloryMon-sem',
       'shapeMon-mean', 'shapeMon-var', 'shapeMon-sem', 'transSizeMon-mean',
       'transSizeMon-var', 'transSizeMon-sem', 'n_segments',
       'phi_c_bulk_round'],
      dtype='object')


In [39]:
cols = ['nmon','dcyl','dcrowd','lcyl','phi_c_bulk_round']
mean_round = lambda x: np.round(np.mean(x),3)
table_1 = equil.groupby(cols)[cols+['fsdMon-mean']].agg({'fsdMon-mean': mean_round})
table_1.reset_index(inplace=True)
table_1['lcyl/fsdMon'] = np.round(table_1['lcyl'] / table_1['fsdMon-mean'],3)
sort_cols = cols + ['lcyl/fsdMon']
table_1.sort_values(by=sort_cols,inplace=True)
table_1.to_csv('table_1.csv',index=False)

In [40]:
cols = ['nmon','dcyl','dcrowd']
mean_round = lambda x: np.round(np.mean(x),3)
table_2 = equil.groupby(cols)[['phi_c_bulk_round']].count()
table_2.reset_index(inplace=True)
#table_2['lcyl/fsdMon'] = np.round(table_1['lcyl'] / table_1['fsdMon-mean'],3)
#sort_cols = cols + ['lcyl/fsdMon']
table_2.sort_values(by=cols,inplace=True)
table_2.to_csv('table_2.csv',index=False)